In [13]:
#if needed...

import import_ipynb
import Eva_and_Status as pi

#copy is needed, sorry ^^
import copy

import networkx as nx
#from cdlib import algorithms, readwrite, evaluation, viz, utils

from collections import defaultdict
from collections import OrderedDict
from collections import Counter
from operator import itemgetter
from itertools import groupby
import operator

#import pandas as pd
import numpy
import json
import csv

from os import listdir
import os.path

#import matplotlib.pyplot as plt
#%matplotlib inline

In [5]:
node_labels = sorted([f for f in listdir('/home/salvatore/Scrivania/NEWnode_labels')]) #label directory
projections = sorted([f for f in listdir('/home/salvatore/Scrivania/Projections')]) #network directory

#to check if they are the same
#print(len(node_labels))
#print(len(projections))

In [6]:
#list of networks
list_g = list()
#for all networks
for i, d in enumerate(projections):
    #build the net
    g = nx.Graph()
    with open('/home/salvatore/Scrivania/Projections/' + d,'rt')as f:
        data = csv.reader(f)
        for l in data:
            l = [s.split() for s in l]
            g.add_edge(l[0][0],l[0][1])        
    list_g.append(g) #add to the list

In [8]:
#directory files are ordered, so it works
for i, nl in enumerate(node_labels):
    with open('/home/salvatore/Scrivania/NEWnode_labels/' + nl,'rt')as f:
        data = csv.reader(f)
        for l in data:
            if l[0] in list_g[i].nodes():
                #add the labels to the nodes
                list_g[i].add_node(l[0], labels=l[1])

In [11]:
#SOME CHECK AND ANALYSIS EXAMPLE

#number of nodes for each net
#for g in list_g:
#    print(g.number_of_nodes())

#labels of the firs net
#print(list_g[0].nodes.data())

In [18]:
for i, g in enumerate(list_g):
    
    #create yourself a path named "Communities_Proj"
    path = "/home/salvatore/Scrivania/Communities_Proj/" + projections[i]
    #create the directorys in "Communities_Proj
    os.mkdir(path)
    
    #IF ALL NODES ARE LABELED, WE DON'T NEED THIS
    temp_g=list()
    for k, v in g.nodes.data():
        if len(v)!=0:
            temp_g.append(k)
    #new graph without no labeled nodes
    new_g = nx.subgraph(g, temp_g)
    
    # nodename -> map to int
    #use g or new_g
    mapping = dict(zip(new_g, range(0, len(new_g))))
    # reverse function
    inv_map = {v: k for k, v in mapping.items()}
    g_relabel = nx.relabel_nodes(new_g, mapping)
    #print(g_relabel.number_of_nodes())
    
    for alpha in [0, 0.7, 0.8, 0.9, 1]:
        #EVA!
        part, com_labels = pi.best_partition(g_relabel, alpha=alpha, beta=1-alpha)
        
        modularity = pi.modularity(part,g_relabel)
        print(modularity)
        purity = pi.purity(com_labels)
        
        #list of list of communities
        sorted_part = sorted(list(part.items()), key=itemgetter(1))
        groups = groupby(sorted_part, key=itemgetter(1))
        group_by_com_attr = [[x[0] for x in v] for k, v in groups]

        #save it with the original namenodes
        original_group = list()
        for el in group_by_com_attr:
            el = copy.deepcopy(el)
            el = nx.subgraph(g_relabel, el)
            el = nx.relabel_nodes(el, inv_map)
            original_group.append(list(el.nodes()))
        
        coms = defaultdict(int)
        for n, c in part.items():
            coms[c] += 1
            
        #print(sorted(dict(coms).items(), key=lambda x: x[0]))
        attr = sorted(com_labels.items(), key=lambda x: x[0])
        
        #the most frequent attr
        max_attrs = list()
        for el in attr:
            max_attrs.append( [el[0], max(el[1].items(), key=operator.itemgetter(1))[0]] )
        
        #save it!
        res = {"coms": original_group,
           "alpha": alpha,
           "purity": purity,
           "modularity": modularity,
           "coms_max_label": max_attrs}
        
        js = json.dumps(res)
        filename = "alpha_" + str(alpha)
        f = open(os.path.join(path, filename), 'w')
        f.write(js)
        f.close()

0.3165046939955119
0.3374064325015338
0.33699004149746414
0.33468462290605305
0.2861127140657349
0.14192894090701763
0.13216098446249552
0.12677963140885667
0.12417387253617493
0.11323647868324262
0.37581089408865537
0.375790593470545
0.37547100761762425
0.3724419038987361
0.3202549455872997
0.4075570759231465
0.40590421187857034
0.40353688325878095
0.4055234634727918
0.36867358812594386
0.0927353357713107
0.09173731101214745
0.09119590905733745
0.09113739798541326
0.07906488577026775
0.17665288858026218
0.1759776953080462
0.17611444743163285
0.17214981087536288
0.15113002125307565
0.2558626206610412
0.2518124372440196
0.25735064643716943
0.25456037360232464
0.22311731410801194
0.296328125
0.283515625
0.288203125
0.288203125
0.25617187500000005
0.4452672588793866
0.4437255964362589
0.445093689661364
0.44222393194234816
0.4071137377765305
0.5
-0.25
-0.25
-0.25
-0.25
0.5
-0.25
-0.25
-0.25
-0.25
0.5017342591266359
0.50113800183841
0.5030531372239438
0.5016872059208758
0.4551050383703407
0